In [1]:
import pandas as pd
from py2neo import Graph


In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "123"), name="llmkg")


In [58]:
def search(keyword):
    q = f"""
    match (n)
    where apoc.text.fuzzyMatch(toLower(n.name), toLower('{keyword}'))
    return ID(n) as id, labels(n) as label, properties(n) as property
    """

    rel = graph.run(q).to_data_frame()
    return rel

In [59]:
keyword = "han-wei shen"
search(keyword)


,id,label,property
0,1456,[Author],"{'name': 'Han-Wei Shen', 'uri': 'bnode://genid..."


In [60]:
def expand(nodeid):
    q = f"""
    match (m)-[p]-(n)
    where id(m) = {idx}
    return ID(n) as id, labels(n) as label, properties(n) as property
    """
    result = graph.run(q).to_data_frame()
    return result

In [61]:
expand(1456).head(5)

,id,label,property
0,27253,[Affiliation],{'uri': 'http://www.wikidata.org/entity/Q309331'}
1,75250,[Paper],"{'volume': '29', 'identifier': '1H1gfbVcpgI', ..."
2,77390,[Paper],"{'identifier': '1LRbR0bikbC', 'issueIdentifier..."
3,78464,[Paper],"{'volume': '29', 'identifier': '1LUpFscWKxq', ..."
4,75255,[Paper],"{'volume': '29', 'identifier': '1H1gjOQxk40', ..."


In [89]:
def filterNode(n_type):
    q= f"""
    match (n: {n_type})
    return ID(n) as id, labels(n) as label, properties(n) as property
    """
    return graph.run(q)

In [92]:
filterNode('data_input')

id,label,property
78,['data_input'],"{name: 'unorganized sample points', uri: 'bnode://genid-b5148842e39a43fc88681588920159b8-ED3BD96F66339E83D54ECC59BBC84BAE'}"
124,['data_input'],"{name: 'sparse elevation data', uri: 'bnode://genid-b5148842e39a43fc88681588920159b8-37C73C576413C6EB82EE4564C6778F78'}"
146,['data_input'],"{name: 'nanoscience simulations', uri: 'bnode://genid-b5148842e39a43fc88681588920159b8-6710B62ABB89064BC64F58A0C3B61BED'}"


In [68]:
def filterEdge(n_type):

    q = f"""
    match (n)-[p: {n_type}]->(m)
    return ID(n) as source_id, labels(n) as source_label, properties(n) as source_property, 
    ID(p) as edge_id, ID(m) as target_id, labels(m) as target_label, properties(m) as target_property

    """
    return graph.run(q).to_data_frame()
    

In [70]:
filterEdge('solved_by').head(4)

,source_id,source_label,source_property,edge_id,target_id,target_label,target_property
0,11751,[background_task],"{'name': 'generating anisotropic samples', 'ur...",49,1701,[method_name],"{'name': 'anisotropic metric', 'uri': 'bnode:/..."
1,18101,[background_task],{'name': 'improving realism of haptic interact...,103,2424,[method_name],"{'name': 'event-based haptic paradigm', 'uri':..."
2,1816,[background_task],{'name': 'modeling natural interaction of clot...,166,1409,[method_name],{'name': 'efficient collision detection method...
3,20079,[background_task],{'name': 'design and evaluation of information...,178,5471,[method_name],"{'name': 'analytic gaps framework', 'uri': 'bn..."


In [71]:
q = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output
"""
graph.run(q)

output
"{properties: [{property: '_handleVocabUris', type: 'INTEGER'}, {property: '_domainRel', type: 'STRING'}, {property: '_keepLangTag', type: 'BOOLEAN'}, {property: '_keepCustomDataTypes', type: 'BOOLEAN'}, {property: '_handleMultival', type: 'INTEGER'}, {property: '_objectPropertyLabel', type: 'STRING'}, {property: '_rangeRel', type: 'STRING'}, {property: '_classLabel', type: 'STRING'}, {property: '_handleRDFTypes', type: 'INTEGER'}, {property: '_subClassOfRel', type: 'STRING'}, {property: '_applyNeo4jNaming', type: 'BOOLEAN'}, {property: '_dataTypePropertyLabel', type: 'STRING'}, {property: '_subPropertyOfRel', type: 'STRING'}], labels: '_GraphConfig'}"
"{properties: [{property: 'uri', type: 'STRING'}, {property: 'name', type: 'STRING'}], labels: 'Author'}"
"{properties: [{property: 'uri', type: 'STRING'}, {property: 'name', type: 'STRING'}], labels: 'background_task'}"


In [76]:
q = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type="RELATIONSHIP" and elementType="relationship"
RETURN label as label, property as source, other[0] as target
"""
graph.run(q).to_data_frame()

,label,source,target
0,uses,method_name,method_technique
1,has_method,Paper,method_name
2,solved_by,background_task,method_name
3,applied_in,background_task,background_application
4,has_evaluation_baseline,Paper,evaluation_baseline
5,evaluated_with,method_name,evaluation_baseline
6,evaluated_by,method_name,evaluation_technique
7,has_evaluation_data,Paper,evaluation_data
8,evaluated_on,method_name,evaluation_data
9,has_technique,Paper,method_technique


In [86]:
q = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE elementType = "node" and type="STRING" and label<>"_GraphConfig"
RETURN label as entity,  property as properties
"""
graph.run(q).to_data_frame()

,entity,properties
0,Author,uri
1,Author,name
2,background_task,uri
3,background_task,name
4,data_output,uri
5,data_output,name
6,data_input,uri
7,data_input,name
8,Affiliation,uri
9,Affiliation,name
